In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (12, 15)
from pyspark.sql import SparkSession
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

hv.notebook_extension()


In [128]:
""" Interface for Data Ingestion.
"""
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


from __future__ import print_function
from __future__ import absolute_import
import os
import numpy as np
import pandas as pd 
import xarray as xr
import itertools
from glob import glob
# from pyspark.sql import SparkSession # Removing this line simply makes the library compatible with Spark 1.6.3 !

def ncread(sc, paths, mode='single', **kwargs):
    """Calls sparkxarray netcdf read function based on the mode parameter.

    ============ ==============================
    Mode          Reading Function
    ------------ ------------------------------
    single       : read_nc_single
    multi        : read_nc_multi
    Anything else: Throw an exception
    ============= ==============================

    Parameters
    ----------

    sc       :  sparkContext object

    paths    :  str or sequence
                Either a string glob in the form "path/to/my/files/*.nc" or an explicit
                list of files to open

    mode     : str
               'single' for a single file
               'multi' for multiple files

    **kwargs : dict
               partitioning options to be passed on to the actual read function.
            
    
    """

    if 'partitions' not in kwargs:
        kwargs['partitions'] = None

    if 'partition_on' not in kwargs:
        kwargs['partition_on'] = ['time']
    
    if 'decode_times' not in kwargs:
        kwargs['decode_times'] = True

    error_msg = ("You specified a mode that is not implemented.")

    if (mode == 'single'):
        return _read_nc_single(sc, paths, **kwargs)

    elif (mode == 'multi'):
        return _read_nc_multi(sc, paths, **kwargs)
    else:
        raise NotImplementedError(error_msg)

        
def _read_nc_single(sc, paths, **kwargs):
    """ Read a single netCDF file

    Parameters
    -----------
    sc       :  sparkContext object

    paths    :  str
                an explicit filename to open
    

    **kwargs : dict
               Additional arguments for partitioning 

    """
    partition_on = kwargs.get('partition_on')
    partitions = kwargs.get('partitions')
    decode_times=kwargs.get('decode_times')

    dset = xr.open_dataset(paths, autoclose=True, decode_times=decode_times)

    # D = {'dim_1': dim_1_size, 'dim_2': dim_2_size, ...}
    D = {dset[dimension].name:dset[dimension].size for dimension in partition_on}
    
    # dim_sizes = [range(dim_1_size), range(dim_2_size), range(...)]
    dim_ranges = [range(dim_size) for dim_size in D.values()]
    

    dim_cartesian_product_indices = [element for element in itertools.product(*dim_ranges)]

    # create a list of dictionaries for  positional indexing
    positional_indices = [dict(zip(partition_on, ij)) for ij in dim_cartesian_product_indices]

    if not partitions:
        partitions = len(dim_cartesian_product_indices)

    if partitions > len(dim_cartesian_product_indices):
        partitions = len(dim_cartesian_product_indices)

    
    # Create an RDD
    rdd = sc.parallelize(positional_indices, partitions).map(lambda x: _readone_slice(dset, x))

    return rdd


def _readone_slice(dset, positional_indices):
    """Read a slice from an xarray.Dataset.

    Parameters
    ----------

    dset                : file_object
                         xarray.Dataset object
    positional_indices  : dict
                          dict containing positional indices for each dimension
                          e.g. {'lat': 0, 'lon': 0}

    Returns
    ---------
    chunk               : xarray.Dataset
                         a subset of the Xarray Dataset

    """

    # Change the positional indices into slice objects
    # e.g {'lat': 0, 'lon': 0} ---> {'lat': slice(0, 1, None),  'lon': slice(0, 1, None)}
    positional_slices = {dim: slice(positional_indices[dim], positional_indices[dim]+1) 
                                                         for dim in positional_indices}

    # Read a slice for the given positional_slices
    chunk = dset[positional_slices]
    return chunk


def _read_nc_multi(sc, paths, **kwargs):
    """ Read multiple netCDF files

    Parameters
    -----------
    sc       :  sparkContext object

    paths    :  str or sequence
                Either a string glob in the form "path/to/my/files/*.nc" or an explicit
                list of files to open

    **kwargs : dict
               Additional arguments for partitioning 

    """

    partition_on = kwargs.get('partition_on')
    partitions = kwargs.get('partitions')

    dset = xr.open_mfdataset(paths, autoclose=True)

    # D = {'dim_1': dim_1_size, 'dim_2': dim_2_size, ...}
    D ={dset[dimension].name:dset[dimension].size for dimension in partition_on}
    
    # dim_sizes = [range(dim_1_size), range(dim_2_size), range(...)]
    dim_ranges = [range(dim_size) for dim_size in D.values()]

    dim_cartesian_product_indices = [element for element in itertools.product(*dim_ranges)]

    # create a list of dictionaries for positional indexing
    positional_indices = [dict(zip(partition_on, ij)) for ij in dim_cartesian_product_indices]

    if not partitions:
        partitions = len(dim_cartesian_product_indices) / 50

    if partitions > len(dim_cartesian_product_indices):
        partitions = len(dim_cartesian_product_indices)

    
    # Create an RDD
    rdd = sc.parallelize(positional_indices, partitions).map(lambda x: readone_slice(dset, x))

    return rdd



In [3]:
# Create sparksession
spark = SparkSession.builder.appName("bias").getOrCreate()
sc = spark.sparkContext

In [167]:
FILE_1 = "/home/abanihi/Documents/Github/spark-xarray/datasets/AFRICA_KNMI-RACMO2.2b_CTL_ERAINT_MM_50km_1989-2008_tasmax.nc"
FILE_2 = "/home/abanihi/Documents/Github/spark-xarray/datasets/AFRICA_UC-WRF311_CTL_ERAINT_MM_50km-rg_1989-2008_tasmax.nc"

In [168]:
knmi = xr.open_dataset(FILE_1, decode_times=False)
knmi

<xarray.Dataset>
Dimensions:       (bnds: 2, height: 1, rlat: 201, rlon: 194, time: 240)
Coordinates:
  * time          (time) float64 468.0 469.0 470.0 471.0 472.0 473.0 474.0 ...
  * rlon          (rlon) float32 -24.64 -24.2 -23.76 -23.32 -22.88 -22.44 ...
    lon           (rlat, rlon) float32 -24.64 -24.2 -23.76 -23.32 -22.88 ...
  * rlat          (rlat) float32 -45.76 -45.32 -44.88 -44.44 -44.0 -43.56 ...
    lat           (rlat, rlon) float32 -45.76 -45.76 -45.76 -45.76 -45.76 ...
  * height        (height) float32 2.0
Dimensions without coordinates: bnds
Data variables:
    rotated_pole  |S1 b''
    time_bnds     (time, bnds) float64 468.0 469.0 469.0 470.0 470.0 471.0 ...
    tasmax        (time, height, rlat, rlon) float64 283.4 283.4 283.4 283.4 ...
Attributes:
    institution:     KNMI
    Conventions:     CF-1.0
    conventionsURL:  http://www.cgd.ucar.edu/cms/eaton/cf-metadata/index.html
    source:          RACMO2.2b
    project_id:      ENSEMBLES
    experiment_id:   ERA

In [169]:
wrf = xr.open_dataset(FILE_2, decode_times=False)
wrf

<xarray.Dataset>
Dimensions:       (bnds: 2, height: 1, rlat: 201, rlon: 194, time: 240)
Coordinates:
    lon           (rlat, rlon) float64 -24.64 -24.2 -23.76 -23.32 -22.88 ...
    lat           (rlat, rlon) float64 -45.76 -45.76 -45.76 -45.76 -45.76 ...
  * height        (height) float32 2.0
  * time          (time) float64 1.426e+04 1.429e+04 1.432e+04 1.435e+04 ...
  * rlat          (rlat) float64 -45.76 -45.32 -44.88 -44.44 -44.0 -43.56 ...
  * rlon          (rlon) float64 -24.64 -24.2 -23.76 -23.32 -22.88 -22.44 ...
Dimensions without coordinates: bnds
Data variables:
    tasmax        (time, height, rlat, rlon) float64 283.4 283.4 283.5 283.5 ...
    rotated_pole  |S1 b''
    time_bnds     (time, bnds) float64 1.424e+04 1.428e+04 1.428e+04 ...
Attributes:
    Conventions:               CF-1.4
    institution:               Universidad de Cantabria (Spain)
    title:                     CORDEX Africa Sensitivity Run
    comment:                   The simulation was forced with E

In [170]:
import ocw.data_source.local as local
import ocw.dataset_processor as dsp
import ocw.evaluation as evaluation
import ocw.metrics as metrics
import ocw.plotter as plotter

In [171]:
""" Step 1: Load Local NetCDF Files into OCW Dataset Objects """
print("Loading %s into an OCW Dataset Object" % (FILE_1,))
knmi_dataset = local.load_file(FILE_1, "tasmax")
print("KNMI_Dataset.values shape: (times, lats, lons) - %s \n" %
      (knmi_dataset.values.shape,))

print("Loading %s into an OCW Dataset Object" % (FILE_2,))
wrf_dataset = local.load_file(FILE_2, "tasmax")
print("WRF_Dataset.values shape: (times, lats, lons) - %s \n" %
      (wrf_dataset.values.shape,))

Loading /home/abanihi/Documents/Github/spark-xarray/datasets/AFRICA_KNMI-RACMO2.2b_CTL_ERAINT_MM_50km_1989-2008_tasmax.nc into an OCW Dataset Object
KNMI_Dataset.values shape: (times, lats, lons) - (240, 201, 194) 

Loading /home/abanihi/Documents/Github/spark-xarray/datasets/AFRICA_UC-WRF311_CTL_ERAINT_MM_50km-rg_1989-2008_tasmax.nc into an OCW Dataset Object
WRF_Dataset.values shape: (times, lats, lons) - (240, 201, 194) 



In [172]:
""" Step 2: Temporally Rebin the Data into an Annual Timestep """
print("Temporally Rebinning the Datasets to an Annual Timestep")
knmi_dataset = dsp.temporal_rebin(knmi_dataset, temporal_resolution='annual')
wrf_dataset = dsp.temporal_rebin(wrf_dataset, temporal_resolution='annual')
print("KNMI_Dataset.values shape: %s" % (knmi_dataset.values.shape,))
print("WRF_Dataset.values shape: %s \n\n" % (wrf_dataset.values.shape,))

Temporally Rebinning the Datasets to an Annual Timestep
KNMI_Dataset.values shape: (20, 201, 194)
WRF_Dataset.values shape: (20, 201, 194) 




In [175]:
""" Step 3: Spatially Regrid the Dataset Objects to a 1 degree grid """
#  The spatial_boundaries() function returns the spatial extent of the dataset
print("The KNMI_Dataset spatial bounds (min_lat, max_lat, min_lon, max_lon) are: \n"
      "%s\n" % (knmi_dataset.spatial_boundaries(), ))
print("The KNMI_Dataset spatial resolution (lat_resolution, lon_resolution) is: \n"
      "%s\n\n" % (knmi_dataset.spatial_resolution(), ))

The KNMI_Dataset spatial bounds (min_lat, max_lat, min_lon, max_lon) are: 
(-45.7599983215332, 42.2400016784668, -24.639999389648438, 60.279998779296875)

The KNMI_Dataset spatial resolution (lat_resolution, lon_resolution) is: 
(0.43999863, 0.44000053)




In [176]:
min_lat, max_lat, min_lon, max_lon = knmi_dataset.spatial_boundaries()

# Using the bounds we will create a new set of lats and lons on 1 degree step
new_lons = np.arange(min_lon, max_lon, 1)
new_lats = np.arange(min_lat, max_lat, 1)

# Spatially regrid datasets using the new_lats, new_lons numpy arrays
print("Spatially Regridding the KNMI_Dataset...")
knmi_dataset = dsp.spatial_regrid(knmi_dataset, new_lats, new_lons)
print("Final shape of the KNMI_Dataset: \n"
      "%s\n" % (knmi_dataset.values.shape, ))

Spatially Regridding the KNMI_Dataset...
Final shape of the KNMI_Dataset: 
(20, 88, 85)



In [177]:
wrf_dataset = dsp.spatial_regrid(wrf_dataset, new_lats, new_lons)
print("Final shape of the WRF_Dataset: \n"
      "%s\n" % (wrf_dataset.values.shape, ))

Final shape of the WRF_Dataset: 
(20, 88, 85)



In [178]:
""" Step 4:  Build a Metric to use for Evaluation - Bias for this example """
# You can build your own metrics, but OCW also ships with some common metrics
print("Setting up a Bias metric to use for evaluation")
bias = metrics.Bias()

Setting up a Bias metric to use for evaluation


In [179]:
""" Step 5: Create an Evaluation Object using Datasets and our Metric """
# The Evaluation Class Signature is:
# Evaluation(reference, targets, metrics, subregions=None)
# Evaluation can take in multiple targets and metrics, so we need to convert
# our examples into Python lists.  Evaluation will iterate over the lists
print("Making the Evaluation definition")
bias_evaluation = evaluation.Evaluation(knmi_dataset, [wrf_dataset], [bias])
print("Executing the Evaluation using the object's run() method")
bias_evaluation.run()

Making the Evaluation definition
Executing the Evaluation using the object's run() method


In [180]:
""" Step 6: Make a Plot from the Evaluation.results """
# The Evaluation.results are a set of nested lists to support many different
# possible Evaluation scenarios.
#
# The Evaluation results docs say:
# The shape of results is (num_metrics, num_target_datasets) if no subregion
# Accessing the actual results when we have used 1 metric and 1 dataset is
# done this way:
print("Accessing the Results of the Evaluation run")
results = bias_evaluation.results[0][0]
print("The results are of type: %s" % type(results))


Accessing the Results of the Evaluation run
The results are of type: <class 'numpy.ma.core.MaskedArray'>


In [181]:
OUTPUT_PLOT = "wrf_bias_compared_to_knmi"

In [182]:
print("Generating a contour map using ocw.plotter.draw_contour_map()")

lats = new_lats
lons = new_lons
fname = OUTPUT_PLOT
gridshape = (4, 5)  # 20 Years worth of plots. 20 rows in 1 column
plot_title = "TASMAX Bias of WRF Compared to KNMI (1989 - 2008)"
sub_titles = range(1989, 2009, 1)

plotter.draw_contour_map(results, lats, lons, fname,
                         gridshape=gridshape, ptitle=plot_title,
                         subtitles=sub_titles)
plt.show()

Generating a contour map using ocw.plotter.draw_contour_map()


In [183]:
print("Making the Evaluation definition")
bias_evaluation = evaluation.Evaluation(wrf_dataset, [knmi_dataset], [bias])
print("Executing the Evaluation using the object's run() method")
bias_evaluation.run()
print("Accessing the Results of the Evaluation run")
results = bias_evaluation.results[0][0]
print("The results are of type: %s" % type(results))


Making the Evaluation definition
Executing the Evaluation using the object's run() method
Accessing the Results of the Evaluation run
The results are of type: <class 'numpy.ma.core.MaskedArray'>


In [184]:
OUTPUT_PLOT = "knmi_bias_compared_to_wrf"

In [ ]:
print("Generating a contour map using ocw.plotter.draw_contour_map()")

lats = new_lats
lons = new_lons
fname = OUTPUT_PLOT
gridshape = (4, 5)  # 20 Years worth of plots. 20 rows in 1 column
plot_title = "TASMAX Bias of KNMI Compared to WRF (1989 - 2008)"
sub_titles = range(1989, 2009, 1)

plotter.draw_contour_map(results, lats, lons, fname,
                         gridshape=gridshape, ptitle=plot_title,
                         subtitles=sub_titles)
plt.show()